In [1]:
import pyxdf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
def load_data(subject, xdf_data_folder, csv_data_folder):
    data, header = pyxdf.load_xdf(f'{xdf_data_folder}/s{subject}.xdf')
    exp_data = (pd.read_csv(f'{csv_data_folder}/{subject}.csv'))
    return data, exp_data

In [3]:
def scatter_triggers(data, tobii, n, b, t):
    all_data = data[tobii]['time_stamps'][2:]
    plt.scatter([all_data[i] for i in range(0, n * b * t, t)][0:10], [1 for _ in range(n)][0:10], marker='o')
    plt.scatter([all_data[i] for i in range(1, n * b * t, t)][0:10], [1 for _ in range(n)][0:10], marker='o')
    plt.scatter([all_data[i] for i in range(2, n * b * t, t)][0:10], [1 for _ in range(n)][0:10], marker='o')
    plt.scatter([all_data[i] for i in range(3, n * b * t, t)][0:10], [1 for _ in range(n)][0:10], marker='o')
    plt.legend(['fixation', 'stim', 'choice', 'outcome'])

In [4]:
def calculate_avg_eye_data(data, p, trial_sizes, subject, num_partitions, marker, tobii, b, n, t, missing_triggers=0):
    # p is the partition inedx 
    x_left_avg = []
    y_left_avg = []
    x_right_avg = []
    y_right_avg = []
    triggers = data[marker]['time_stamps'][missing_triggers:]
    fixation_trigers = [triggers[i] for i in range(0, n * t * b, t)]
    choice_trigers = [triggers[i] for i in range(2, n * t * b, t)]
    for i in range(b * n):  # number of trials * number of blocks
        eye_time = data[tobii]['time_stamps']
        flag = 1
        for j in range(len(eye_time)):   
            if eye_time[j] > fixation_trigers[i] and flag:
                first = j
                flag = 0
            elif eye_time[j] > choice_trigers[i]:
                last = j
                flag = 1
                break
        eye_data = data[tobii]['time_series']
        partition = int(np.round((last - first) / num_partitions))
        trial_sizes[subject].append(last - first)
        x_left_avg.append(np.nanmean(eye_data[0:,23][first + p*partition:first + (p+1)*partition]))
        y_left_avg.append(np.nanmean(eye_data[0:,24][first + p*partition:first + (p+1)*partition]))
        x_right_avg.append(np.nanmean(eye_data[0:,25][first + p*partition:first + (p+1)*partition]))
        y_right_avg.append(np.nanmean(eye_data[0:,26][first + p*partition:first + (p+1)*partition]))
    return x_left_avg, y_left_avg, x_right_avg, y_right_avg

In [5]:
def set_final_data(exp_data, x_left_avg, y_left_avg, x_right_avg, y_right_avg, b, n):
    d = {'label':[], 'x_left':[], 'y_left':[], 'x_right':[],'y_right':[]}
    choice = exp_data[' key_ID'].to_numpy()
    for idx in range(n * b):
        if np.isnan(x_left_avg[idx]) or np.isnan(y_left_avg[idx]) or np.isnan(x_right_avg[idx]) or np.isnan(y_right_avg[idx]):
            continue
        d['label'].append(choice[idx])
        d['x_left'].append(x_left_avg[idx])
        d['y_left'].append(y_left_avg[idx])
        d['x_right'].append(x_right_avg[idx])
        d['y_right'].append(y_right_avg[idx])
    return pd.DataFrame(data=d)

In [6]:
def save_final_data(final_data, p, folder, subject):
    final_data.to_csv(f'{folder}/{subject}pre_{p}.csv')

In [7]:
# run once for all subjects
trial_sizes = {}
trial_sizes_avg = {}

In [ ]:
subject = '006'
marker = 1
tobii = 0
n = 100 # num of trials
t = 4 # num of triggers in each trial
b = 1 # num of blocks
missing_triggers = 0
num_partitions = 5
csv_data_folder = 'final_exp_decoding'
xdf_data_folder = 'Data'
trial_sizes[subject] = []

In [ ]:
# main loop for preprocessing eyes data - x, y for each eye.

data, exp_data = load_data(subject, xdf_data_folder, csv_data_folder)
# scatter_triggers(data, tobii, n, b, t)
for p in range(num_partitions):
    x_left_avg, y_left_avg, x_right_avg, y_right_avg = calculate_avg_eye_data(data, p, trial_sizes, subject,
                                                                              num_partitions, marker, tobii, b, 
                                                                              n, t, missing_triggers)
    final_data = set_final_data(exp_data, x_left_avg, y_left_avg, x_right_avg, y_right_avg, b, n)
    save_final_data(final_data, p, csv_data_folder, subject)
trial_sizes_avg[subject] = np.average(trial_sizes[subject])

In [ ]:
# run once for all subjects
pd.DataFrame(data=trial_sizes_avg, index=[1]).to_csv("trial_sizes_avg.csv")